In [ ]:
# resnet은 층이 깊다. 

In [2]:
import tensorflow as tf

In [ ]:
# convolution는 이미지 크기와 상관이 없다. 
# convolution 하면 dense가 함께 붙었다. 이는 이미지 shape를 맞춰줘야한다.

# dense가 없으면 공간정보를 잘 활용할 수 있다. 

In [3]:
# input pipeline 
train_dataset = tf.data.Dataset.list_files(PATH+'train/*.jpg')
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(1)

# 이 구조로 만들어놓으면 편하기 때문에 이렇게 만들어 놓는다. 
# tensorflow, numpy 

NameError: name 'PATH' is not defined

In [4]:
# fully connected 구조 
OUTPUT_CHANNELS = 3


def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [5]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

NameError: name 'inp' is not defined

In [6]:
# 이미지 컨볼루션 했던거 다시 찾아주는거 conv2DTranspose -> 업샘플링은 아님
# 업샘플링은단순히 크기만 키워주는것
# conv2dtranspose는 복원시키는 것 

def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU()) 

  return result

In [7]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

NameError: name 'down_result' is not defined

In [ ]:
# 층이 깊어도 학습이 되거나 오버피팅이 안된다. 
# 오통인코더 관련된 부분에서 배웠던 인코딩하는 것들은 층이 별로 안많았다. 
# 오토인코딩은 딥러닝 나오기부터 나왔던 모델 
# 오버피팅되는거 막기 위해서 리키렐루를 썼다. 
# 배치 노말라이제이션 도 씀. (오버피팅 막기위해서)

def Generator():
  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate() # 사실상 연산자 이름이다. 
# 레이어가 연산하는 애 있다. 

  inputs = tf.keras.layers.Input(shape=[None,None,3])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1]) # 맨 처음과 맨 마지막이 달라서 맨 마지막 레이터를 뺐다. 

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [8]:
generator = Generator()

gen_output = generator(inp[tf.newaxis,...], training=False)
plt.imshow(gen_output[0,...])

NameError: name 'Generator' is not defined

In [ ]:
# tf.keras.layers.Concatenate()()
# tf.keras.layers.concatenate()
# 대문자는 클래스이므로 인스턴스해야한다. 따라서 인스턴스하고 괄호한번 더하기보다 소문자를 쓴다. 

In [10]:
# discriminator 2개를 input으로 받는다. 
# sequence로 모델 2개 받을 수 없다. 

def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[None, None, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[None, None, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)
# 파이썬에서 대문자 클래스/ 소문자 함수 

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)
# zero padding 쓰는 이유 2
# 1.싸이즈 맞추기
# 2.구석 가장자리에 있는애 특성 뽑아내기 위해
  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [11]:
discriminator = Discriminator()
disc_out = discriminator([inp[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

NameError: name 'inp' is not defined

In [12]:
LAMBDA = 100

In [13]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [15]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss

In [16]:
# fit을 하면 gan은 안좋다. 
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints (Object-based saving)
에폭 한번 돌때마다 저장하는 것 ** 중요

- 콜백은 한 에폭마다 저장한다. 
- 배치에 따라서 저장하려면 하는 방법이 다르다. 

In [ ]:
# computational graph 

In [17]:
EPOCHS = 150

In [18]:
def generate_images(model, test_input, tar):
  # the training=True is intentional here since
  # we want the batch statistics while running the model
  # on the test dataset. If we use training=False, we will get
  # the accumulated statistics learned from the training dataset
  # (which we don't want)
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [19]:
# 동시에 같이 썼다. 
# @tf.funtion 이거하면 컴파일 되서 속도 최적화 할 수 있다. 
@tf.function
def train_step(input_image, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

In [20]:
def fit(train_ds, epochs, test_ds): 
    # 에폭 안에는 배치까지 고려할 수 있다. 
  for epoch in range(epochs):
    start = time.time()

    # Train
    for input_image, target in train_ds:
      train_step(input_image, target)

    clear_output(wait=True)
    # Test on the same image so that the progress of the model can be 
    # easily seen.
    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))

In [21]:
fit(train_dataset, EPOCHS, test_dataset)

NameError: name 'train_dataset' is not defined

In [22]:
# 현실세계는 짝을 이루는 데이터가 잘 없다. 
# 짝이 없어도 새로운 애를 생성하는 것을 

1 2 3 4 5 6 7 8


In [29]:
digits = list(map(int, input().split(' ')))
ascending = [1,2,3,4,5,6,7,8]
descending = [ 8,7,6,5,4,3,2,1]
if digits == ascending:
    print('ascending')
elif digits == descending:
    print('descending')
else:
    print('mixed')

ascending


## cycle GAN
- stype transfer 
- pix2pix

neural style 현대적인 방법이 cycle GAN이다. 

In [30]:
import tensorflow as tf

In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds

In [38]:
dir(tfds)
# 연습용 데이터
dir(tfds.image.cycle_gan)

['CycleGAN',
 'CycleGANConfig',
 '_CITATION',
 '_DATA_OPTIONS',
 '_DL_URL',
 '_DL_URLS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'api_utils',
 'division',
 'os',
 'print_function',
 'tf',
 'tfds']

In [35]:
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [42]:
tfds.image.CycleGAN.data_dir

In [ ]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=False, as_supervised=True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

C:\Users\Cho Gyung Ah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
# unet 기반으로 generate 했다. 

In [ ]:
# 대문자: 클래스, 소문자: 함수
# 다 대문자: 람다 (함수 만드는 것)

In [ ]:
# fit - batch size : callback 이 에폭 시점에서 나온다. 
# train on batch : 배치사이즈에서 callback를 불러올 수 있다. 

# tf.train
# keras.proprocessing input(vgg, ,...) 이름은 똑같지만 사용하는 용도는 다르다. 

In [ ]:
# dense 다 빼고나서 글로벌 average pooling 

# trainable -> false선택하면 웨이트 업데이트를 안한다. 

In [ ]:
%matplotlib inline

from os import listdir 
from numpy import asarray
from numpy import vstack 
from tensorflow.keras.preprocessing.image import img_to_array

# 넘파이로 불러오면 편하다 
# fit 
# fit_generator (사이즈를 알아서 맞춰준다.) 

In [1]:
import tensorflow as tf

In [2]:
tf.image.flip_left_right()
# 단점: 데이터 지원하는 기능이 아직까지는 작다. 

TypeError: flip_left_right() missing 1 required positional argument: 'image'

In [4]:
import IPython.display as display 

In [5]:
# 텐서는 .numpy붙이면 넘파이로 바꿀 수 있지만, 거꾸로는 안될 수도 있따. 
# 넘파이에서 텐서로 가는건 어려울 수 있음 

In [7]:
# pil 
# standardization 하면 빨라진다. 

In [ ]:
# train, test 할때마다 랜덤하게 나눈다. 
# 데이터가 많으면 많을수록 그런 경향이 줄어든다. 
# 가끔 문제가 생길 때가 있다. 

In [ ]:
# style transfer 는 한부분만 짤라서 하는게 아니고 전체를 convolution 자체를 스타일 자체로 만드는 구조로 바꿔버려서..? pix2pix만 쓸 수 있다. (컨텐츠 구조로 만드는 부분, 스타일 만드는 구조)
# 짝만들기 위해서 얼룩말 pix2pix 